### Añadimos SRC a la raíz del proyecto para poder importar el contenido

In [ ]:
import sys
import os

# Obtener la ruta absoluta de la carpeta raíz (donde está src)
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Subir un nivel desde notebooks/

# Agregar la carpeta src al path
sys.path.append(os.path.join(ROOT_DIR, "src"))

### Importamos los módulos necesarios

In [ ]:
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import json

from config import VICTORY_ROAD_SCRAPPING_DATA_PATH

### Hacemos Scraping en la página web de Victory Road y sacamos información sobre los Worlds de los últimos 3 años

In [ ]:
# Diccionario con URLs de los torneos mundiales
VGC_WORLDS_URLS = {
    "Worlds2024_Usage": "https://worldcupvgc.com/es/stats/pokemon",
    "Worlds2023_Usage": "https://2023.worldcupvgc.com/es/stats/pokemon",
    "Worlds2022_Usage": "https://2022.worldcupvgc.com/es/stats/pokemon"
}

# Lista para almacenar los datos
worlds_data = []

# Función para extraer datos de cada torneo
def scrape_worlds_vgc_data():
    for year, url in VGC_WORLDS_URLS.items():
        print(f"🔎 Accediendo a {year}: {url}")

        try:
            # Hacer la petición HTTP
            response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
            response.raise_for_status()
            soup = BeautifulSoup(response.text, "html.parser")

            # Buscar la etiqueta con JSON embebido
            script_tag = soup.find("script", {"id": "__NEXT_DATA__"})
            
            if not script_tag:
                print(f"⚠️ No se encontró el JSON en {year}.")
                continue
            
            # Convertir el contenido del script en un diccionario JSON
            data = json.loads(script_tag.string)
            
            # Extraer la lista de Pokémon desde la clave "pokemonStats"
            pokemon_stats = data["props"]["pageProps"]["pokemonStats"]

            for p in pokemon_stats:
                pokemon_name = p["name"].strip().lower().replace(" ", "-")
                usage_percent = p["gamesPlayedPercentage"]  # % de uso en torneos
                
                # Guardamos los datos en la lista
                worlds_data.append({"Pokemon": pokemon_name, year: usage_percent})

        except requests.exceptions.RequestException as e:
            print(f"❌ Error al acceder a {url}: {e}")
        
        time.sleep(1)  # Pequeña pausa para evitar bloqueos

# Ejecutar el scraping
scrape_worlds_vgc_data()

# Convertir los datos a DataFrame
df_worlds_vgc = pd.DataFrame(worlds_data)

# Agrupar por Pokémon (algunos pueden repetirse en diferentes ediciones)
df_worlds_vgc = df_worlds_vgc.groupby("Pokemon", as_index=False).first()

# Rellenar valores nulos con "NoUsage"
df_worlds_vgc.fillna("NoUsage", inplace=True)

# Guardar en CSV
df_worlds_vgc.to_csv(VICTORY_ROAD_SCRAPPING_DATA_PATH, index=False)
print("✅ Datos de uso en torneos mundiales guardados en '{VICTORY_ROAD_SCRAPPING_DATA_PATH}'")

# 🔎 Mostrar los primeros datos extraídos
print(df_worlds_vgc.head())


🔎 Accediendo a Worlds2024_Usage: https://worldcupvgc.com/es/stats/pokemon
🔎 Accediendo a Worlds2023_Usage: https://2023.worldcupvgc.com/es/stats/pokemon
🔎 Accediendo a Worlds2022_Usage: https://2022.worldcupvgc.com/es/stats/pokemon
✅ Datos de uso en torneos mundiales guardados en 'worlds_vgc_usage.csv'
                             Pokemon Worlds2024_Usage Worlds2023_Usage  \
0                          abomasnow          NoUsage         0.186567   
1                         aerodactyl          NoUsage          NoUsage   
2                             aggron          NoUsage          NoUsage   
3                           alcremie          NoUsage          NoUsage   
4  alcremie-caramel-swirl-strawberry          NoUsage          NoUsage   

  Worlds2022_Usage  
0          NoUsage  
1         0.096562  
2          0.07725  
3         0.038625  
4         0.038625  


C:\Users\furio\AppData\Local\Temp\ipykernel_6024\2872672488.py:63: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NoUsage' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_worlds_vgc.fillna("NoUsage", inplace=True)
